# T81 Notebook 01 — Balanced Ternary Playground

This notebook is a **deeper dive** into balanced ternary arithmetic.

It builds on `00_t81_orientation.ipynb` and focuses on:

1. Working **directly on balanced ternary digits** (not just via Python ints).
2. Visualizing **carries** and **sign behavior** in `{-1, 0, +1}` space.
3. Sketching how small values can be **promoted** into larger T81-style representations (e.g., towards T243 / T729).

> Canonical semantics live in the C++ implementation and the specs.  
> This notebook is a conceptual mirror and a playground for intuition.


## 0. Imports and Basic Conventions

We reuse the same symbolic convention as in notebook 00:

- `-1  → "-"`
- ` 0  → "0"`
- `+1  → "+"`

Here we will:

- Represent balanced ternary values as **lists of digits** (least-significant first).
- Provide helpers to convert between digit lists and string/int representations.


In [ ]:
from typing import List, Tuple

BT_DIGITS = (-1, 0, 1)
BT_SYMBOLS = {
    -1: "-",
    0: "0",
    1: "+",
}
BT_SYMBOL_TO_VALUE = {v: k for k, v in BT_SYMBOLS.items()}

def int_to_bt_digits(n: int) -> List[int]:
    """Convert an int to a list of balanced ternary digits, LSB-first.

    Example: 2 → [-1, 1]  (because 2 = -1 + 1*3)
    """
    if n == 0:
        return [0]

    digits: List[int] = []
    x = n
    while x != 0:
        x, r = divmod(x, 3)
        if r == 2:
            r = -1
            x += 1
        digits.append(r)
    return digits

def bt_digits_to_int(digits: List[int]) -> int:
    """Convert a list of digits (LSB-first) back to an int."""
    value = 0
    for d in reversed(digits):
        value = value * 3 + d
    return value

def bt_digits_to_str(digits: List[int]) -> str:
    return "".join(BT_SYMBOLS[d] for d in reversed(digits))

def bt_str_to_digits(s: str) -> List[int]:
    return [BT_SYMBOL_TO_VALUE[ch] for ch in reversed(s)]

# Quick sanity check
for n in range(-5, 6):
    ds = int_to_bt_digits(n)
    s = bt_digits_to_str(ds)
    back = bt_digits_to_int(ds)
    print(f"{n:3d} → {s:>4s} (digits={ds}) → {back:3d}")


## 1. Digit-wise Balanced Ternary Addition

In notebook 00, we implemented `bt_add` by converting to `int`, adding, and converting back.

Here, we implement **addition directly in digit space**, mirroring the kind of logic a VM might use:

1. Align digits LSB-first.
2. Add digit-wise plus a carry.
3. Normalize the result into `{-1, 0, +1}` with a new carry.

We also expose the intermediate carries for debugging and intuition.


In [ ]:
def normalize_bt_sum(raw: int) -> Tuple[int, int]:
    """Normalize a raw sum of two digits + carry into (digit, new_carry).

    raw can range from -3 to +3.
    We want digit ∈ {-1, 0, +1} and carry adjusted accordingly.
    """
    if raw <= -2:
        # e.g., raw = -2 → digit = +1, carry = -1
        return raw + 3, -1
    elif raw >= 2:
        # e.g., raw = +2 → digit = -1, carry = +1
        return raw - 3, 1
    else:
        # raw ∈ {-1, 0, +1}
        return raw, 0

def bt_add_digits(a: List[int], b: List[int]) -> Tuple[List[int], List[int]]:
    """Add two BT digit lists (LSB-first).

    Returns (result_digits, carry_trace) where carry_trace records the carry
    *entering* each position (for visualization).
    """
    max_len = max(len(a), len(b))
    result: List[int] = []
    carry_trace: List[int] = []
    carry = 0

    for i in range(max_len):
        da = a[i] if i < len(a) else 0
        db = b[i] if i < len(b) else 0
        carry_trace.append(carry)
        raw = da + db + carry
        digit, carry = normalize_bt_sum(raw)
        result.append(digit)

    if carry != 0:
        carry_trace.append(carry)
        result.append(carry)

    # Trim any leading zeros from the most significant side
    while len(result) > 1 and result[-1] == 0:
        result.pop()

    return result, carry_trace

def bt_add_str(a: str, b: str) -> str:
    da = bt_str_to_digits(a)
    db = bt_str_to_digits(b)
    res, _ = bt_add_digits(da, db)
    return bt_digits_to_str(res)

# Compare digit-wise addition vs int-based addition
examples = [(3, 4), (5, -2), (-3, -4), (7, 7)]
print("a   b   →  bt(a)   bt(b)   bt(a+b)   int(a+b)")
for a, b in examples:
    sa = bt_digits_to_str(int_to_bt_digits(a))
    sb = bt_digits_to_str(int_to_bt_digits(b))
    sc = bt_add_str(sa, sb)
    print(f"{a:2d} {b:3d} → {sa:>5s}  {sb:>5s}  {sc:>7s}   {a+b:3d}")


### 1.1 Visualizing Carry Behavior

The next helper prints a small, human-readable trace of the addition process:

- per-digit operands
- raw sums
- normalized digits
- carries


In [ ]:
def print_bt_add_trace(a: int, b: int) -> None:
    da = int_to_bt_digits(a)
    db = int_to_bt_digits(b)
    res, carry_trace = bt_add_digits(da, db)

    print(f"Adding {a} + {b}")
    print(f"  a digits (LSB→): {da}")
    print(f"  b digits (LSB→): {db}")
    print(f"  carry in:        {carry_trace}")
    print(f"  result digits:   {res}")
    print(f"  result bt:       {bt_digits_to_str(res)}")
    print(f"  result int:      {bt_digits_to_int(res)}")

print_bt_add_trace(5, -3)
print()
print_bt_add_trace(-4, -7)


## 2. Balanced Ternary as a Substrate for T81 Types

In the C++ implementation, T81 types like `T81Int`, `T81BigInt`, `T81Fraction`, and higher layers (T243, T729) are built on top of a carefully specified numeric substrate.

Here, we sketch a **toy representation** of a small T81-style integer as:

- A sign (implicitly reflected in the digits).
- A vector of balanced ternary digits.

> This is *not* the actual C++ layout or ABI. It is a conceptual way to see how balanced ternary digits can be grouped into higher-level types.


In [ ]:
class ToyT81Int:
    """A minimal, toy T81-style integer backed by balanced ternary digits.

    This is purely for intuition and experimentation.
    """

    def __init__(self, value: int):
        self._digits = int_to_bt_digits(value)

    @property
    def value(self) -> int:
        return bt_digits_to_int(self._digits)

    def __str__(self) -> str:
        return bt_digits_to_str(self._digits)

    def __repr__(self) -> str:
        return f"ToyT81Int(bt='{str(self)}', value={self.value})"

    def __add__(self, other: "ToyT81Int") -> "ToyT81Int":
        res_digits, _ = bt_add_digits(self._digits, other._digits)
        res = ToyT81Int(0)
        res._digits = res_digits
        return res

# Example
x = ToyT81Int(10)
y = ToyT81Int(-4)
z = x + y
print("x =", x)
print("y =", y)
print("z = x + y =", z, "=", z.value)


## 3. Where This Connects to the C++ Implementation

To see how these ideas are realized in the actual T81 codebase:

- Read: `spec/t81-data-types.md` for the formal definition of T81/T243/T729.
- Inspect: `include/t81/core/T81Int.hpp` and related headers.
- Look at the tests under `tests/` that exercise `T81Int` and, when present, `T81BigInt`.

You can use this notebook to:

- Prototype edge cases.
- Build intuition about carries and sign behavior.
- Compare toy results against the actual C++ behavior for small values.
